In [2]:
import os 
import struct
import logging
import argparse
import enum

# logging.basicConfig(level=logging.WARNING)  # 正常模式
logging.basicConfig(level=logging.DEBUG)  # 调试模式

In [4]:
file_path = r'E:\workspace\parse_bitstream\parse_bitstream\data\system_wrapper_1_原版.bit'

In [19]:
bit_head_content = b''
start_index = 0
end_index = 0
with open(file_path, 'rb') as f:
# ========== 起始数据 ============
    bit_head_content += f.read(2) # 标识
    bit_head_content += f.read(11) # 内容
    start_index += 13
    # ========== 起始数据 ============
    
    # ========== 标识符A ============
    
    bit_head_content += f.read(1) # 标识 b'\x61'
    cur_group_length = f.read(2)
    bit_head_content += cur_group_length  # 内容 该组字节长度
    group_length = struct.unpack('>H', cur_group_length)[0]
    bit_head_content += f.read(group_length) # 文件路径、Version、UserID
    start_index += 3
    end_index = start_index + group_length
    logging.debug(f"\tA content: {bit_head_content[start_index:end_index].hex()}")
    start_index = end_index
    # ========== 标识符A ============
    
    # ========== 标识符B ============
    bit_head_content += f.read(1) # 标识 b'\x62'
    cur_group_length = f.read(2)
    bit_head_content += cur_group_length  # 内容 该组字节长度
    group_length = struct.unpack('>H', cur_group_length)[0]
    bit_head_content += f.read(group_length) # part name
    start_index += 3
    end_index = start_index + group_length
    logging.debug(f"\tB content: {bit_head_content[start_index:end_index].hex()}")
    start_index = end_index
    # ========== 标识符B ============
    
    # ========== 标识符C ============
    bit_head_content += f.read(1) # 标识 b'\x63'
    cur_group_length = f.read(2)
    bit_head_content += cur_group_length  # 内容 该组字节长度
    group_length = struct.unpack('>H', cur_group_length)[0]
    bit_head_content += f.read(group_length) # 年/月/日
    start_index += 3
    end_index = start_index + group_length
    logging.debug(f"\tC content: {bit_head_content[start_index:end_index].hex()}")
    start_index = end_index
    # ========== 标识符C ============

    # ========== 标识符D ============
    bit_head_content += f.read(1) # 标识 b'\x64'
    cur_group_length = f.read(2)
    bit_head_content += cur_group_length  # 内容 该组字节长度
    group_length = struct.unpack('>H', cur_group_length)[0]
    bit_head_content += f.read(group_length) # 时:分:秒
    start_index += 3
    end_index = start_index + group_length
    logging.debug(f"\tD content: {bit_head_content[start_index:end_index].hex()}")
    start_index = end_index
    # ========== 标识符D ============

    # ========== 标识符E ============
    bit_head_content += f.read(1) # 标识 b'\x65'
    bit_head_content += f.read(4) # 位流的总长度
    start_index += 1
    end_index = start_index + 4
    logging.debug(f"\tE content: {bit_head_content[start_index:end_index].hex()}")
    start_index = end_index
    # ========== 标识符E ============
    
    for i in range(65):
        bit_cfg_content = f.read(4)
    
    test_content = struct.unpack('>I', bit_cfg_content)[0]
    
    

DEBUG:root:	A content: 6578616d706c655f746f703b5573657249443d305846464646464646463b56657273696f6e3d323032302e3200
DEBUG:root:	B content: 37613130307466676734383400
DEBUG:root:	C content: 323032342f30372f303900
DEBUG:root:	D content: 31313a31363a323100
DEBUG:root:	E content: 003a607c


In [13]:
class OpCode(enum.Enum):
    NOOP = 0
    READ = 1
    WRITE = 2
    
class Address(enum.Enum):
    CRC = 0
    FAR = 1
    FDRI = 2
    FDRO = 3
    CMD = 4
    CTL0 = 5
    MASK = 6
    STAT = 7
    LOUT = 8
    COR0 = 9
    MFWR = 10
    CBC = 11
    IDCODE = 12 
    AXSS = 13
    COR1 = 14
    UNKNOWN_15 = 15 #if write, csob_reg(ib) <= packet, csbo_cnt(ib) <= word count, csbo_flag(ib) < '1'
    WBSTAR = 16
    TIMER = 17
    UNKNOWN_18 = 18
    POST_CRC = 19 #Undocumented in UG470
    UNKNOWN_20 = 20
    UNKNOWN_21 = 21
    BOOTSTS = 22
    CTL1 = 24
    UNKNOWN_30 = 30 #if next packet is Type2 and bcout_cnt(ib) = 0, set bocut_flag(ib) <= '1' and bout_cnt(ib) <= word count
    BSPI = 31

In [17]:
def get_packet_type(word):
    return word >> 29 

def get_type_1_packet_content(word):
    header_type = get_packet_type(word)
    opcode = OpCode((word >> 27) & 0x3)
    address = Address((word >> 13) & 0x1F)
    reserved = (word >> 11) & 0x3
    word_count = word & 0x7FF
    return {
            "header_type":header_type,
            "opcode":opcode,
            "address":address,
            "reserved":reserved,
            "word_count":word_count
        }

In [21]:
a = get_type_1_packet_content(test_content)

In [24]:
a["opcode"] == OpCode.WRITE

True

In [11]:
# 001 10 000000000 00010 00 00000000000

0

In [9]:
test_content

187